In [1]:
!pip install pandas openpyxl transformers torch
!pip install llama-cpp-python


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from llama_cpp import Llama
from concurrent.futures import ThreadPoolExecutor

model_path = "/Users/briannapatten/Desktop/mistral-7b-instruct-v0.1.Q4_K_M.gguf"
llm = Llama(model_path=model_path)

llama_model_load_from_file_impl: using device Metal (Apple M2 Pro) - 10922 MiB free
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/briannapatten/Desktop/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:          

In [4]:

def translate_query(query):
    prompt = f"### Instruction:\nRephrase the following ESGish query into a concise natural English sentence. These queries return a set of issuers from a larger dataset. With this in mind can you also generate a variety of natural language openers prior to the associated prompts to form the full English sentence.\n\n{query}\n\n### Response:"    
    response = llm(prompt, max_tokens=575, temperature=0.1)
    return response["choices"][0]["text"].strip()
query = "[CEOSpecialGrantPct] > '0.5'"

In [5]:
import pandas as pd
from functools import partial
import time

df = pd.read_excel("esgish_short.xlsx")
queries = df["Esgish"].tolist()
batch_size = 100  
translated_queries = []

for i in range(0, len(queries), batch_size):
    batch = queries[i:i + batch_size]
    batch_translations = [translate_query(q) for q in batch] 
    translated_queries.extend(batch_translations)
    print(f"Processed {i + len(batch)} / {len(queries)} queries")
    if (i // batch_size + 1) % 10 == 0:
        print(f"Processed {i + len(batch)} / {len(queries)} queries")
    time.sleep(1)

df["English"] = translated_queries
df.to_excel("translated_queries.xlsx", index=False)

llama_perf_context_print:        load time =    6430.83 ms
llama_perf_context_print: prompt eval time =    6429.81 ms /   129 tokens (   49.84 ms per token,    20.06 tokens per second)
llama_perf_context_print:        eval time =    1314.81 ms /    23 runs   (   57.17 ms per token,    17.49 tokens per second)
llama_perf_context_print:       total time =    7750.23 ms /   152 tokens
Llama.generate: 77 prefix-match hit, remaining 13 prompt tokens to eval
llama_perf_context_print:        load time =    6430.83 ms
llama_perf_context_print: prompt eval time =     464.50 ms /    13 tokens (   35.73 ms per token,    27.99 tokens per second)
llama_perf_context_print:        eval time =     902.96 ms /    18 runs   (   50.16 ms per token,    19.93 tokens per second)
llama_perf_context_print:       total time =    1370.39 ms /    31 tokens
Llama.generate: 74 prefix-match hit, remaining 9 prompt tokens to eval
llama_perf_context_print:        load time =    6430.83 ms
llama_perf_context_print: pr

Processed 19 / 19 queries
